In [1]:
import torch
from tqdm import tqdm

from gfos.model.gnn import LayoutModel
from gfos.data.dataset import LayoutDataset, Normalizer
from gfos.data.utils import load_layout

In [2]:
LAYOUT_DIR = r"H:\data\gfos\predict-ai-model-runtime\npz_all\npz\layout"
NORMALIZER_PATH = "../../data/normalizer.json"

SOURCE = "nlp"
SEARCH = "default"

In [3]:
normalizer = Normalizer.from_json(NORMALIZER_PATH, SOURCE, SEARCH)

layout_data = load_layout(
    LAYOUT_DIR,
    model_type=SOURCE,
    compile_type=SEARCH,
)

test_dataset = LayoutDataset(
    layout_data["valid"],
    normalizer=normalizer,
)

Loading data:   0%|          | 0/20 [00:00<?, ?it/s]

Loading data: 100%|██████████| 20/20 [00:28<00:00,  1.43s/it, talking-heads_large_batch_size_16_train]                       


In [4]:
model = LayoutModel(node_feat_dim=112, node_config_dim=14, config_neighbor_layer="GATConv", dropout=0.2)
state_dict = torch.load(r"G:\projects\gfos\logs\pipeline\nlp_ensemble\multiruns\2023-10-27_10-58-40\0\wandb\run-20231027_110222-0ytzj3l1\files\1679_0.9513.pth")
model.load_state_dict(state_dict["state_dict"])
model.eval().cuda();

In [12]:
for layer in model.dense.children():
    if hasattr(layer, "reset_parameters"):
        print(f"Resetting {layer}")
        layer.reset_parameters()

Resetting Linear(in_features=64, out_features=64, bias=False)
Resetting Linear(in_features=64, out_features=64, bias=False)
Resetting Linear(in_features=64, out_features=1, bias=False)


In [13]:
# model1 = LayoutModel(node_feat_dim=112, node_config_dim=14, config_neighbor_layer="GATConv", dropout=0.2)
# state_dict = torch.load(r"G:\projects\gfos\logs\pipeline\layout_nlp_default_20231024_020455\runs\2023-10-24_02-04-55\wandb\run-20231024_020902-b7lcp2vw\files\1119_0.4991.pth")
# model1.load_state_dict(state_dict["state_dict"])

# model2 = LayoutModel(node_feat_dim=112, node_config_dim=14, config_neighbor_layer="GATConv", dropout=0.2)
# state_dict = torch.load(r"G:\projects\gfos\logs\pipeline\layout_nlp_default_20231017_114957\runs\2023-10-17_11-49-57\wandb\run-20231017_115319-2el0weqq\files\1239_0.5009.pth")
# model2.load_state_dict(state_dict["state_dict"])


Disable config edge weight
Disable config edge weight


<All keys matched successfully>

In [1]:
model

NameError: name 'model' is not defined

In [14]:
model1 = model1.eval().cuda()
model2 = model2.eval().cuda()
models = [model1, model2]

In [5]:
from gfos.metrics import LayoutMetrics

In [6]:
device = "cuda"
batch_size = 100

results = {}
metric = LayoutMetrics()

with torch.no_grad():
    for record in tqdm(test_dataset):
        model_id = record["model_id"]
        node_feat = record["node_feat"]
        node_opcode = record["node_opcode"]
        edge_index = record["edge_index"]
        node_config_feat = record["node_config_feat"]
        node_config_ids = record["node_config_ids"]
        config_runtime = record["config_runtime"]
        config_edge_index = record["config_edge_index"]
        config_edge_weight = record["config_edge_weight"]

        (
            node_feat,
            node_opcode,
            edge_index,
            node_config_feat,
            node_config_ids,
            config_edge_index,
            config_edge_weight,
        ) = (
            node_feat.to(device),
            node_opcode.to(device),
            edge_index.to(device),
            node_config_feat.to(device),
            node_config_ids.to(device),
            config_edge_index.to(device),
            config_edge_weight.to(device),
        )

        c = len(config_runtime)

        outs = []
        # for model in models:
            # outs = []
            
        for i in range(0, c, batch_size):
            end_i = min(i + batch_size, c)
            out: torch.Tensor = model(
                node_feat,
                node_opcode,
                edge_index,
                node_config_feat[i:end_i],
                node_config_ids,
                config_edge_index,
            )
            outs.append(out.detach().cpu())
        outs = torch.cat(outs)
            # blend_outs.append(outs)
        # outs = torch.stack(blend_outs).mean(dim=0)
        metric.add(model_id, outs.numpy(), config_runtime.numpy())
        results[model_id] = torch.argsort(outs).cpu().numpy()

100%|██████████| 20/20 [03:27<00:00, 10.38s/it]


In [8]:
metric.compute_scores()

{'raw_kendall': 0.4696834004353601,
 'index_kendall': 0.4696658273174156,
 'top100_error': 0.998,
 'top500_error': 0.9839000000000002,
 'kendall_albert_en_xlarge_batch_size_16_test': 0.49129285977373977,
 'kendall_bert_en_cased_L-12_H-768_A-12_batch_size_16_test': 0.5492576655600119,
 'kendall_bert_multi_cased_L-12_H-768_A-12_batch_size_16_train': 0.44564061571086705,
 'kendall_small_bert_bert_en_uncased_L-10_H-128_A-2_batch_size_32_test': 0.5683349755090917,
 'kendall_small_bert_bert_en_uncased_L-10_H-128_A-2_batch_size_64_train': 0.2971721520149192,
 'kendall_small_bert_bert_en_uncased_L-10_H-256_A-4_batch_size_32_test': 0.5492520412041304,
 'kendall_small_bert_bert_en_uncased_L-10_H-256_A-4_batch_size_64_train': 0.3701117356517411,
 'kendall_small_bert_bert_en_uncased_L-10_H-512_A-8_batch_size_64_test': 0.40175130499584977,
 'kendall_small_bert_bert_en_uncased_L-10_H-768_A-12_batch_size_16_train': 0.44908342283413644,
 'kendall_small_bert_bert_en_uncased_L-10_H-768_A-12_batch_size_3

In [20]:
with open(f"../../output/{SOURCE}_{SEARCH}.csv", "w") as f:
    f.write("ID,TopConfigs\n")
    for k, v in results.items():
        model_id = f"layout:{SOURCE}:{SEARCH}:"+k
        values = ";".join([str(i) for i in v])
        f.write(f"{model_id},{values}\n")
